<a href="https://colab.research.google.com/github/iamdsc/deep_learning/blob/master/12_keras_cats_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Download the dataset for cats and dogs
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2019-03-15 15:11:03--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.27.180.156, 2600:1402:f000:48e::e59, 2600:1402:f000:497::e59
Connecting to download.microsoft.com (download.microsoft.com)|184.27.180.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M  66.8MB/s    in 8.0s    

2019-03-15 15:11:11 (98.9 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [0]:
!unzip kagglecatsanddogs_3367a.zip

In [0]:
# importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [0]:
%matplotlib inline
DATADIR = 'PetImages'
CATEGORIES = ['Dog', 'Cat']
IMG_SIZE = 50

# building the training data
training_data = []

def create_training_data():  
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category) # path to cat or dogs dir
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass

In [0]:
create_training_data()
print(len(training_data))

24946


In [0]:
# shuffle the data
import random

random.shuffle(training_data)

In [0]:
X=[]
y=[]

for features, label in training_data:
  X.append(features)
  y.append(label)

In [0]:
# converting feature list into numpy array
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [0]:
# saving the pre-processed dataset
import pickle

pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [0]:
# Load the data from pickle file
X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

# normalize the data
X = X/255.0

In [0]:
# building the model
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# train the model
model.fit(X, y, batch_size=32, validation_split=0.1, epochs=15)